# EarthRanger IO

## Setup

### Ecoscope

In [1]:
!pip install ecoscope &> /dev/null

In [2]:
import getpass
import json
import os
import sys

import geopandas as gpd
import pandas as pd
from erclient.client import ERClientException
from shapely.geometry import Point

import ecoscope

ecoscope.init()

/home/gitonga/anaconda3/envs/ecoscope/lib/python3.8/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_77657/1215752612.py:6: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest

 _____
|   __|___ ___ ___ ___ ___ ___ ___
|   __|  _| . |_ -|  _| . | . | -_|
|_____|___|___|___|___|___|  _|___|
                          |_|



### Google Drive Setup

In [ ]:
output_dir = "Ecoscope-Outputs"

if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive/", force_remount=True)
    output_dir = os.path.join("/content/drive/MyDrive/", output_dir)

os.makedirs(output_dir, exist_ok=True)

### Connect to EarthRanger

In [12]:
ER_SERVER = os.getenv("ER_SERVER", "https://mep-dev.pamdas.org")
ER_USERNAME = os.getenv("ER_USERNAME", "gitonga")

ER_PASSWORD = os.getenv("ER_PASSWORD", "")

if not ER_PASSWORD:
    ER_PASSWORD = getpass.getpass("Please enter your ER password: ")

er_io = ecoscope.io.EarthRangerIO(
    server=ER_SERVER,
    username=ER_USERNAME,
    password=ER_PASSWORD,
    tcp_limit=5,
    sub_page_size=4000,
)

Please enter your ER password:  ········


## GET `Event`

### By `event_ids`
Use this approach to download an individual `Event` if you know its id. Downloading from multiple ids at once is currently unsupported in ER.

In [ ]:
er_io.get_events(
    event_ids="855903a6-dfab-4b91-b75d-c993bea1208d",  # Elephant sighting event
)

### By `event_type`
Use this approach if you want to download events of a given event type (using the EventType ID from ER)

In [ ]:
er_io.get_events(
    event_type=["518bcc0a-fd6a-4607-a066-56f716720b0c"],  # MEP Elephant Sighting
    since=pd.Timestamp("2019-05-01").isoformat(),
    until=pd.Timestamp("2020-05-01").isoformat(),
)

### Unpack `event_details` column of `JSON` data into individual columns

In [ ]:
events_df = er_io.get_events(event_type=["e00ce1f6-f9f1-48af-93c9-fb89ec493b8a"])  # MEP Distance Survey Count
events_df.event_details

In [ ]:
er_io._normalize_column(events_df, "event_details")
events_df

## POST `Event`

First delete any existing events:

In [ ]:
event_df = er_io.get_events(
    event_type=["b0d66402-34cb-4e9a-9715-2ac4eeebb574"],  # Carcass
)

for i, value in event_df.iterrows():
    er_io.delete_event(event_id=event_df.loc[i, "id"])

Define events to be uploaded:

In [ ]:
new_event = [
    {
        "event_details": {
            "carcassrep_ageofanimal": None,
            "carcassrep_ageofcarcass": None,
            "carcassrep_causeofdeath": None,
            "carcassrep_sex": None,
            "carcassrep_species": None,
            "carcassrep_trophystatus": None,
        },
        "event_type": "carcass_rep",
        "icon_id": "carcass_rep",
        "id": "e29f9078-ee0a-4f06-b685-92e9ff266e9b",
        "location": {"latitude": -27.12759, "longitude": -109.40804},
        "priority": 100,
        "state": "resolved",
        "time": pd.Timestamp("2022-03-15 15:44:00-0700"),
        "title": "Carcass",
    }
]

Read events as dataframe:

In [ ]:
new_event_df = pd.DataFrame.from_dict(new_event)
new_event_df

Upload:

In [ ]:
try:
    er_io.post_event(events=new_event_df.iloc[[1]])
except ERClientException as e:
    print(e)

## PATCH `Event`

Define updated values

In [ ]:
updated_event = pd.DataFrame(
    [
        {
            "event_details": {
                "carcassrep_ageofanimal": "adult",
                "carcassrep_ageofcarcass": "less_than_a_day",
                "carcassrep_causeofdeath": "unnaturalshot",
                "carcassrep_sex": "female",
                "carcassrep_species": "elephant",
                "carcassrep_trophystatus": "removed",
            }
        }
    ]
)

Get and display current values

In [ ]:
event_df = er_io.get_events(
    event_type=["b0d66402-34cb-4e9a-9715-2ac4eeebb574"],  # Carcass
)

event_df.loc[0, "event_details"]

Patch and confirm values have been updated

In [ ]:
id_to_patch = event_df.loc[0, "id"]
er_io.patch_event(event_id=id_to_patch, events=updated_event).loc[0, "event_details"]

## GET `Subjects`

### by `id`

In [ ]:
er_io.get_subjects(id="64444ed7-72ec-4531-a2b1-fb25c7197b2d")  # Habiba

### By `SubjectGroup` `group_name`

In [ ]:
er_io.get_subjects(group_name="Elephants")

## GET `Observations`

The output is an Ecoscope `Relocations` dataframe that has a core set of columns: `groupby_col`, `fixtime`, `junk_status`.

All other columns are prepended by `extra__`. The `groupby_col` will be the `subject_id` and the index of the dataframe is the observation indices.

Classic ER `Observation` format can be returned by passing in parameter `relocations=False`.

Filter observation by setting `filter` to one of: None, 0, 1, 2 or 3
- None returns everything
- 0 filters out everything but observations with exclusion flag 0 (Pass back clean data)
- 1 filters out everything but observations with exclusion flag 1 (Pass back manually filtered data)
- 2 filters out everything but observations with exclusion flag 2 (Ppass back automatically filtered data)
- 3 filters out everything but observations with exclusion flag 2 or 1 (Pass back both manual and automatic filtered data)

### By `SubjectGroup` `group_name`:

In [ ]:
since = pd.Timestamp("2008-01-01").isoformat()
until = pd.Timestamp("2013-01-01").isoformat()

In [ ]:
relocs = er_io.get_subjectgroup_observations(
    group_name="Elephants",
    filter=0,
    since=since,
    until=until,
    include_details=True,
    include_inactive=True,
    include_source_details=True,
    include_subject_details=True,
    # include_subjectsource_details=True
)
relocs

### By `subject_id`

In [ ]:
relocs = er_io.get_subject_observations(
    subject_ids=[
        "64444ed7-72ec-4531-a2b1-fb25c7197b2d",  # Habiba
        "b8be28f7-8c20-46d9-85a5-fd817351bde5",  # Salif Keita
    ],
    since=since,
    until=until,
    filter=0,
    include_details=True,
    include_subject_details=True,
    include_source_details=True,
    # include_subjectsource_details=True,
)
relocs

### By `source_id`

In [ ]:
relocs = er_io.get_source_observations(
    source_ids=[
        "5e6ce0f4-5deb-47b4-a65b-539f00dc716d",  # Habiba's Source
        "ea18dc70-8bcf-4eb5-8329-a0716f309fa5",  # Salif Keita's Source
    ],
    since=since,
    until=until,
    filter=0,
    include_details=True,
    include_source_details=True,
)
relocs

## GET `Patrols`

In [13]:
er_io.get_patrols(
    patrol_type=["0ef3bf48-b44c-4a4e-a145-7ab2e38c9a57"] # MEP Distance Count Survey
)  

,id,priority,state,objective,serial_number,title,files,notes,patrol_segments,updates
0,2c54c31b-8dff-4bf8-8d2a-94a619077e2a,100,done,None,20349,None,[],[],[{'id': '6b4d20b3-663b-407a-9926-33659ecc9634'...,"[{'message': 'Patrol Added', 'time': '2023-05-..."
1,e4457241-b70c-491c-876b-d4d26434df1d,100,done,None,20350,None,[],[],[{'id': '8b6f4310-a59f-4be4-8fa3-593a87fd1ee9'...,"[{'message': 'Patrol Added', 'time': '2023-05-..."
2,8c516f60-f5c9-456d-84a5-bd46e9b76452,100,done,None,20351,None,[],[],[{'id': 'fa54a990-fa4e-4896-aebb-1e92166c26c8'...,"[{'message': 'Patrol Added', 'time': '2023-05-..."
3,7296c2d2-0cef-4c4e-a84b-723d8947ad5a,100,done,None,20352,None,[],[],[{'id': 'baadcf5d-ca75-419e-bb23-b9ef0c2d8c0b'...,"[{'message': 'Patrol Added', 'time': '2023-05-..."
4,354b1049-9a98-44e5-b7d5-0fcaebed8cef,100,done,None,20353,None,[],[],[{'id': '9974f7b1-4133-43d3-bd13-ee83d0e32910'...,"[{'message': 'Patrol Added', 'time': '2023-05-..."
5,5d4e75b3-d8b4-49a7-8c22-71daaae35870,100,done,None,20354,None,[],[],[{'id': 'da17f1d7-e07d-4e13-8770-be7c348c554b'...,"[{'message': 'Patrol Added', 'time': '2023-05-..."
6,f85d30dc-3eab-4508-89ca-077cf9424ff5,100,done,None,20355,None,[],[],[{'id': '831fcb48-3b75-4785-8f1f-66cf05f2fade'...,"[{'message': 'Patrol Added', 'time': '2023-05-..."
7,5d370004-8a2a-4ae6-a93b-0e57f3335df0,100,done,None,20356,None,[],[],[{'id': '8b908cf1-851e-4cc8-aed6-fe0d4cb77b85'...,"[{'message': 'Patrol Added', 'time': '2023-05-..."
8,154aa1f3-b72c-4b82-b4f8-6864a8a9875f,100,done,None,20357,None,[],[],[{'id': 'c68278ab-9ebb-470d-926d-66d2c222d6da'...,"[{'message': 'Patrol Added', 'time': '2023-05-..."
9,434a537b-863d-45e8-9196-b4b902fcb4fb,100,done,None,20358,None,[],[],[{'id': '469edbe5-7daa-4e65-8bdd-666b8c0cebf4'...,"[{'message': 'Patrol Added', 'time': '2023-05-..."


## GET `Observations` for a `Patrol`

In [ ]:
patrol_df = er_io.get_patrols()

relocs = er_io.get_observations_for_patrols(
    patrol_df,
    include_source_details=False,
    include_subject_details=False,
    include_subjectsource_details=False,
)

## GET `Users`

In [ ]:
df = pd.DataFrame(er_io.get_users())
df

## POST `Observation`

Upload observations for existing source 

In [ ]:
observations = [
    {
        "fixtime": pd.Timestamp.utcnow().isoformat(),
        "geometry": Point(0, 0),
        "source_id": "5e6ce0f4-5deb-47b4-a65b-539f00dc716d",  # Habiba's source
    },
    {
        "fixtime": pd.Timestamp("2023-02-01").isoformat(),
        "geometry": Point(1.345, 2.3303),
        "source_id": "5e6ce0f4-5deb-47b4-a65b-539f00dc716d",  # Habiba's source
    },
    {
        "fixtime": pd.Timestamp("2023-03-21").isoformat(),
        "geometry": Point(3.4455, -3.4554),
        "source_id": "ea18dc70-8bcf-4eb5-8329-a0716f309fa5",  # Salif Keita's source
    },
]

gdf = gpd.GeoDataFrame.from_dict(observations)
gdf

Post:

In [ ]:
er_io.post_observations(observations=gdf, source_id_col="source_id", recorded_at_col="fixtime")

## POST `SubjectSource`

In [ ]:
er_io.post_subjectsource(
    subject_id="b8be28f7-8c20-46d9-85a5-fd817351bde5",  # Salif Keita
    source_id="ea18dc70-8bcf-4eb5-8329-a0716f309fa5",  # Salif Keita's source
    lower_bound_assigned_range=pd.Timestamp.utcnow().isoformat(),
    upper_bound_assigned_range=(pd.Timestamp.utcnow() + pd.Timedelta(days=30)).isoformat(),
    additional={},
)

## Export to File

### Create test data

Skip this cell if you would like to provide your own data

In [ ]:
relocs = er_io.get_subjectgroup_observations(
    group_name="Elephants",
    filter=0,
    since=since,
    until=until,
    include_details=True,
    include_inactive=True,
    include_source_details=True,
    include_subject_details=True,
    # include_subjectsource_details=True
)

### GeoPackage (.gpkg) 
(GeoPackage does not support columns with type `list` so we `drop` them.)

In [ ]:
relocs.drop(
    columns=relocs.columns[relocs.applymap(lambda x: isinstance(x, list)).any()],
    errors="ignore",
    inplace=True,
)

relocs.to_file(os.path.join(output_dir, "easter_island.gpkg"), layer="easter_island")

### CSV

In [ ]:
relocs.to_csv(os.path.join(output_dir, "observations.csv"))

### Python Pickle (.pkl)

In [ ]:
relocs.to_pickle(os.path.join(output_dir, "observations.pkl"))

### Apache Feather (.feather)

In [ ]:
relocs.to_feather(os.path.join(output_dir, "observations.feather"))